# Real time app

## 0. Requirements

- Captura de rostros (capture_faces.py)
Ejecuta un script que utiliza la webcam para capturar imágenes faciales de una persona.
Las imágenes se almacenan en la carpeta Data/NombrePersona/.
- Entrenamiento del modelo LBPH (train_lbph.py)
Procesa las imágenes capturadas y entrena un modelo de reconocimiento facial usando el algoritmo LBPH (Local Binary Patterns Histograms).
El modelo entrenado se guarda como modeloLBPHFace.xml.
- Modelo yolov8n-widerface entrenado best.pt (yolov8n_widerface.ipynb)

## 1. Execution

In [ ]:
# Librerías necesarias
from ultralytics import YOLO
import cv2
import os
import numpy as np
import imutils


In [ ]:
# Ruta donde se encuentran los rostros capturados por persona
dataPath = '../lbph_data/faces'
imagePaths = sorted(os.listdir(dataPath))  # Asume que el orden de carpetas es el mismo que se usó en el entrenamiento

# Cargar modelo entrenado de reconocimiento facial LBPH
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.read('../lbph_data/lbph_model.xml')  # Ruta al modelo LBPH entrenado

# Cargar modelo YOLOv8 entrenado para detección de rostros
model = YOLO('../yolov8n-widerface-v1/best.pt')  # Debe ser tu best.pt o modelo fine-tuned


In [ ]:
# Iniciar cámara
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # Usa webcam
# Alternativamente: cap = cv2.VideoCapture('video.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = imutils.resize(frame, width=640)
    auxFrame = frame.copy()

    # Detección con YOLO
    results = model(frame, verbose=False)
    for box in results[0].boxes.xyxy:
        x1, y1, x2, y2 = map(int, box)

        rostro = auxFrame[y1:y2, x1:x2]
        if rostro.size == 0:
            continue

        rostro_gray = cv2.cvtColor(rostro, cv2.COLOR_BGR2GRAY)
        rostro_resized = cv2.resize(rostro_gray, (150, 150), interpolation=cv2.INTER_CUBIC)

        # Reconocimiento facial
        result = face_recognizer.predict(rostro_resized)

        # Mostrar nombre
        if result[1] < 70:
            nombre = imagePaths[result[0]]
            color = (0, 255, 0)
        else:
            nombre = 'Desconocido'
            color = (0, 0, 255)

        cv2.putText(frame, f'{nombre} ({int(result[1])})', (x1, y1 - 10), 2, 0.8, color, 1, cv2.LINE_AA)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

    cv2.imshow('Reconocimiento Facial', frame)
    if cv2.waitKey(1) == 27:
        break  # ESC para salir

cap.release()
cv2.destroyAllWindows()
